In [2]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%pylab inline
pylab.rcParams['figure.figsize'] = (24, 12)
import numpy as np
import pandas as pd
from scipy import stats
import json
import os
from stldecompose import decompose, forecast
from statsmodels.tsa.seasonal import seasonal_decompose
from elasticsearch import Elasticsearch
import datetime

import importlib
detector = importlib.import_module('thermometr')

Populating the interactive namespace from numpy and matplotlib


In [3]:
client = Elasticsearch()
query_total_bucket = {
  "aggs": {
    "my_aggregation": {
      "date_histogram": {
        "field":     "@timestamp",
        "interval":  "30m"
      }
    }
  }
}

response = client.search(
    index="movi*",
    size=0,
    body=query_total_bucket)

print("Query time: ", response['took'])

#nota: hay que buscar la mejor manera de hacer esto porque parece rebuscado
dates = []
count = []
for element in response['aggregations']['my_aggregation']['buckets']:
    dates.append(pd.to_datetime(element['key_as_string']))
    count.append(element['doc_count'])
    
ts_resampled = pd.Series(count, index=dates)

Query time:  0


KeyError: 'aggregations'

In [ ]:
stl = decompose(ts_resampled, period=336)
stl.seasonal.head(336).plot();

In [ ]:
normaldecomp = seasonal_decompose(ts_resampled, freq=336)
normaldecomp.seasonal.head(336).plot();
                                  

In [ ]:
dif = normaldecomp.seasonal.head(336).sub(stl.seasonal.head(336))
dif.plot()

In [ ]:
det = detector.Thermometr(ts_resampled)
res = det.detect()
print("\n Cantidad de anomalias: " + str(len(res)) + "\n" )
res


In [ ]:
x, y = [], []
for dic_item in res:
    if dic_item['ESD'] > 0.05:
        x.append(dic_item['index'])
        y.append(dic_item['value'])
    
#plt.scatter(x,y, color='blue')
#plt.show()

In [ ]:
ts_resampled.plot(linestyle='-', markevery=x, marker='o', markersize=10, markerfacecolor='b')

In [ ]:
for dic_item in res:
    print(ts_resampled.index[dic_item['index']].value // 10 ** 6)

In [ ]:
type(res[0]['index'])